In [1]:
from astropy.table import vstack, Table
import numpy as np
import pandas as pd
import sqlite3
import data_process_utils as dpu
from astropy import units as u
from astropy.coordinates import SkyCoord
from lsst.afw.table import BaseCatalog
from lsst.daf.persistence import Butler

In [2]:
calexp_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-calexp-v1'
calexp_butler = Butler(calexp_repo)


In [3]:
base_flags = ['base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_saturatedCenter', 'base_PixelFlags_flag_suspect',
              'base_PixelFlags_flag_suspectCenter', 'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge',
              'base_PixelFlags_flag_bad', 'base_NaiveCentroid_flag']

#### host full table

In [7]:
psf_table = Table.read('./data/table/gal_4639/psf_fwhm.csv', format='ascii.csv')

matched_table = Table()
for row in psf_table:
    diff_diaSrc = BaseCatalog.readFits(row['diaSrc_path'])
    diff_diaSrc_astropy = diff_diaSrc.asAstropy()
    # diff_diaSrc_astropy = dpu.remove_flag_astropy(diff_diaSrc_astropy, flag_list=base_flags)
    diff_diaSrc_astropy['diaSrc_snr'] = diff_diaSrc_astropy['base_PsfFlux_instFlux'] / diff_diaSrc_astropy['base_PsfFlux_instFluxErr']
    
    patch = str(row['patch'])
    patch = patch.replace(',', '')
    visit = int(row['visit'])
    det = int(row['detector'])
    filt = str(row['filter'])
    
    calexp_id = {'visit': visit, 'detector': det, 'filter': filt}
    calexp_src = calexp_butler.get('src', calexp_id)
    src_astropy = calexp_src.asAstropy()
    artifact_coord = SkyCoord(np.rad2deg(diff_diaSrc_astropy['coord_ra'].data),
                              np.rad2deg(diff_diaSrc_astropy['coord_dec'].data),
                              frame="icrs", unit="deg")
    src_coord = SkyCoord(np.rad2deg(src_astropy['coord_ra'].data),
                         np.rad2deg(src_astropy['coord_dec'].data),
                         frame="icrs", unit="deg")

    is_artifact, _ = dpu.two_direction_skymatch(src_coord, artifact_coord, radius=2 * u.arcsec)
    
    # match src to fake
    conn = sqlite3.connect('./data/patch_0to6/diff/al_default/detection/detection.sqlite')
    query = f"SELECT * FROM fake_src WHERE fake_mag = '20' AND host_mag = '20_21' AND patch = '{patch}' AND visit = {visit} AND detector = {det} And filter = '{filt}'"
    fake_src_df = pd.read_sql_query(query, conn)
    fake_astropy = Table.from_pandas(fake_src_df)
    fake_coord = SkyCoord(np.rad2deg(fake_astropy['coord_ra'].data),
                          np.rad2deg(fake_astropy['coord_dec'].data),
                          frame="icrs", unit="deg")
    is_fake, _ = dpu.two_direction_skymatch(src_coord, fake_coord, radius=2 * u.arcsec)
    
    host_snr = src_astropy['base_PsfFlux_instFlux'] / src_astropy['base_PsfFlux_instFluxErr']
    t = Table()
    t['host_snr'] = host_snr
    t['is_artifact'] =  is_artifact
    t['is_fake'] = is_fake
    
    matched_table = vstack([matched_table, t])
    
matched_table.write('./data/table/gal_4639/full_hosts.csv', format='ascii.csv', overwrite=True)

#### host full table, base flag applied

In [10]:

psf_table = Table.read('./data/table/gal_4639/psf_fwhm.csv', format='ascii.csv')

matched_table = Table()
for row in psf_table:
    diff_diaSrc = BaseCatalog.readFits(row['diaSrc_path'])
    diff_diaSrc_astropy = diff_diaSrc.asAstropy()
    diff_diaSrc_astropy = dpu.remove_flag_astropy(diff_diaSrc_astropy, flag_list=base_flags)
    diff_diaSrc_astropy['diaSrc_snr'] = diff_diaSrc_astropy['base_PsfFlux_instFlux'] / diff_diaSrc_astropy['base_PsfFlux_instFluxErr']
    
    patch = str(row['patch'])
    patch = patch.replace(',', '')
    visit = int(row['visit'])
    det = int(row['detector'])
    filt = str(row['filter'])
    
    calexp_id = {'visit': visit, 'detector': det, 'filter': filt}
    calexp_src = calexp_butler.get('src', calexp_id)
    src_astropy = calexp_src.asAstropy()
    artifact_coord = SkyCoord(np.rad2deg(diff_diaSrc_astropy['coord_ra'].data),
                              np.rad2deg(diff_diaSrc_astropy['coord_dec'].data),
                              frame="icrs", unit="deg")
    src_coord = SkyCoord(np.rad2deg(src_astropy['coord_ra'].data),
                         np.rad2deg(src_astropy['coord_dec'].data),
                         frame="icrs", unit="deg")
    is_artifact, _ = dpu.two_direction_skymatch(src_coord, artifact_coord, radius=2 * u.arcsec)
    
    # match src to fake
    conn = sqlite3.connect('./data/patch_0to6/diff/al_default/detection/detection.sqlite')
    query = f"SELECT * FROM fake_src WHERE fake_mag = '20' AND host_mag = '20_21' AND patch = '{patch}' AND visit = {visit} AND detector = {det} And filter = '{filt}'"
    fake_src_df = pd.read_sql_query(query, conn)
    fake_astropy = Table.from_pandas(fake_src_df)
    fake_coord = SkyCoord(np.rad2deg(fake_astropy['coord_ra'].data),
                          np.rad2deg(fake_astropy['coord_dec'].data),
                          frame="icrs", unit="deg")
    is_fake, _ = dpu.two_direction_skymatch(src_coord, fake_coord, radius=2 * u.arcsec)
    
    host_snr = src_astropy['base_PsfFlux_instFlux'] / src_astropy['base_PsfFlux_instFluxErr']
    t = Table()
    t['host_snr'] = host_snr
    t['is_artifact'] =  is_artifact
    t['is_fake'] = is_fake
    
    matched_table = vstack([matched_table, t])
    
matched_table.write('./data/table/gal_4639/full_hosts_base_applied.csv', format='ascii.csv', overwrite=True)


#### artifact full table

In [7]:
"""
psf_table = Table.read('./data/table/gal_4639/psf_fwhm.csv', format='ascii.csv')

matched_table = Table()
for row in psf_table:
    diff_diaSrc = BaseCatalog.readFits(row['diaSrc_path'])
    diff_diaSrc_astropy = diff_diaSrc.asAstropy()
    visit = int(row['visit'])
    det = int(row['detector'])
    filt = str(row['filter'])
    calexp_id = {'visit': visit, 'detector': det, 'filter': filt}
    calexp_src = calexp_butler.get('src', calexp_id)
    src_astropy = calexp_src.asAstropy()
    
    artifact_coord = SkyCoord(np.rad2deg(diff_diaSrc_astropy['coord_ra'].data),
                              np.rad2deg(diff_diaSrc_astropy['coord_dec'].data),
                              frame="icrs", unit="deg")
    src_coord = SkyCoord(np.rad2deg(src_astropy['coord_ra'].data),
                         np.rad2deg(src_astropy['coord_dec'].data),
                         frame="icrs", unit="deg")
    matched_status, matched_idx = dpu.two_direction_skymatch(artifact_coord, src_coord, radius=2 * u.arcsec)
    
    matched_src = src_astropy[matched_idx]
    matched_snr = matched_src['base_PsfFlux_instFlux'] / matched_src['base_PsfFlux_instFluxErr']
    diff_diaSrc_astropy['host_base_PsfFlux_instFlux'] = matched_src['base_PsfFlux_instFlux']
    diff_diaSrc_astropy['host_base_PsfFlux_instFluxErr'] = matched_src['base_PsfFlux_instFluxErr']
    diff_diaSrc_astropy['host_snr'] = matched_snr
    diff_diaSrc_astropy['matched_status'] = matched_status
    for col in row.columns:
        diff_diaSrc_astropy[col] = [row[col] for i in range(len(diff_diaSrc_astropy))]
    matched_table = vstack([matched_table, diff_diaSrc_astropy])
matched_table.write('./data/table/gal_4639/full_detections.csv', format='ascii.csv', overwrite=True)
""";